In [9]:
from script.utils import chunks
from docx import Document
from docx.shared import Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
import subprocess
import pandas as pd
import numpy as np
import os
import shutil
from tqdm import tqdm
import glob
from zipfile import ZipFile 
from pypdf import PdfWriter

In [10]:
df = pd.read_excel('Dummy DASS 21_AZ.xlsx')
df.head()

,Nama,DoB,Tanggal,SkorDepresi,InterpretasiDepresi,SkorAnsietas,Interpretasi Ansieta,SkorStres,InterpretasiStres
0,Andi,1992-04-15,12/08/2025,12,Sedang,8,Ringan,14,Sedang
1,Budi,1989-11-02,13/08/2025,5,Ringan,12,Sedang,10,Ringan
2,Citra,1998-07-21,13/08/2025,18,Berat,15,Sedang,20,Berat
3,Dian,2000-03-09,14/08/2025,7,Ringan,6,Ringan,8,Ringan
4,Eka,1987-12-30,14/08/2025,22,Sangat Berat,19,Berat,25,Sangat Berat


In [19]:
pd.options.mode.chained_assignment = None 

import os
from datetime import datetime
now = datetime.now().strftime('%Y%m%d%H%M%S')
dir_nm = f'MH_Output/{now}'

if not os.path.exists(dir_nm):
    os.makedirs(dir_nm)

for _, row in df.iterrows():
    data = row.copy()
    # Replace NaN with Empty String
    for i in data.keys():
        if str(data.get(i)) == 'nan':
            data[i] = ''
        else:
            data[i] = str(data[i])
    
    # Date Format
    if 'Tanggal' in data and data['Tanggal'] != '':
        data['Tanggal'] = pd.to_datetime(data['Tanggal']).strftime('%d-%m-%Y')
    
    # Open template
    with open('Template DASS-21 - AZ.docx', 'rb') as f:
        doc = Document(f)
    
    ## User Detail
    doc.paragraphs[1].text = doc.paragraphs[1].text.replace('<<Nama>>', data['Nama']).replace('<<Tanggal>>', data['Tanggal'])
    
    ## Score
    doc.tables[1].cell(1,1).text = doc.tables[1].cell(1,1).text.replace('<<SkorDepresi>>', data['SkorDepresi'])
    doc.tables[1].cell(1,2).text = doc.tables[1].cell(1,2).text.replace('<<InterpretasiDepresi>>', data['InterpretasiDepresi'])
    doc.tables[1].cell(2,1).text = doc.tables[1].cell(2,1).text.replace('<<SkorAnsietas>>', data['SkorAnsietas'])
    doc.tables[1].cell(2,2).text = doc.tables[1].cell(2,2).text.replace('<<InterpretasiAnsietas>>', data['Interpretasi Ansieta'])
    doc.tables[1].cell(3,1).text = doc.tables[1].cell(3,1).text.replace('<<SkorStres>>', data['SkorStres'])
    doc.tables[1].cell(3,2).text = doc.tables[1].cell(3,2).text.replace('<<InterpretasiStres>>', data['InterpretasiStres'])
    
    doc.tables[1].cell(1,1).paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.tables[1].cell(1,2).paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.tables[1].cell(2,1).paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.tables[1].cell(2,2).paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.tables[1].cell(3,1).paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.tables[1].cell(3,2).paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
    nm = data['Nama']
    dob = data['DoB']
    tmp_docx = f'{dir_nm}/{nm}_{dob}_MHResult.docx'
    
    doc.save(tmp_docx)
    
    
    # Convert to PDF
    subprocess.run([
        'libreoffice', '--convert-to', 'pdf', tmp_docx, '--outdir', dir_nm
    ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    os.remove(tmp_docx)
    pdf_name = tmp_docx.replace('.docx', '.pdf')
    

/tmp/ipykernel_31662/3551607793.py:22: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['Tanggal'] = pd.to_datetime(data['Tanggal']).strftime('%d-%m-%Y')
/tmp/ipykernel_31662/3551607793.py:22: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['Tanggal'] = pd.to_datetime(data['Tanggal']).strftime('%d-%m-%Y')
/tmp/ipykernel_31662/3551607793.py:22: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['Tanggal'] = pd.to_datetime(data['Tanggal']).strftime('%d-%m-%Y')
/tmp/ipykernel_31662/3551607793.py:22: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to sil

In [ ]:
def pdf_protect(pdf_file):
    reader = PdfReader(pdf_file)
    writer = PdfWriter()

    passw = pdf_file.split('_')[-1].replace('.pdf', '')
    new_fname = '_'.join(pdf_file.split('_')[:-1]) + '.pdf'

    for page in reader.pages:
        writer.add_page(page)

    # Set password (user password)
    writer.encrypt(user_password=passw)

    with open(new_fname, "wb") as f:
        writer.write(f)
    os.remove(pdf_file)
    return new_fname


def upload_and_get_link(self, gl):
    BUCKET_NAME=os.environ['BUCKET_NAME']
    
    s3 = boto3.client(
        "s3",
        region_name="ap-southeast-3",
        aws_access_key_id=os.environ['ACCESS_KEY_ID'],
        aws_secret_access_key=os.environ['SECRET_ACCESS_KEY'] 
    )
    
    # Upload the file
    #for gl in files_gl:  
    fname =  gl.replace('.tmp_gl/', '')
    fname = fname.replace(' ', '_').replace('%', '_').replace("'", '_').replace("+", '_').replace('"', '_')
    #s3.upload_file(gl, BUCKET_NAME, fname)
    
    #Read the PDF file in binary mode
    with open(gl, 'rb') as pdf_file:
        s3.put_object(
            Bucket=BUCKET_NAME,
            Key=fname,  # destination in S3
            Body=pdf_file,
            ContentType='application/pdf',
            ACL="public-read"
        )

    return fname


In [ ]:
import streamlit as st
import boto3
import os
import pandas as pd
from io import BytesIO
from dotenv import load_dotenv
from botocore.exceptions import NoCredentialsError

# Load environment variables
load_dotenv()
# AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID")
# AWS_SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
# AWS_BUCKET_NAME = os.getenv("AWS_BUCKET_NAME")
# AWS_REGION = os.getenv("AWS_REGION", "ap-southeast-1")

def upload_to_s3(file):
    BUCKET_NAME=os.environ['BUCKET_NAME']
    
    s3 = boto3.client(
        "s3",
        region_name="ap-southeast-3",
        aws_access_key_id=os.environ['ACCESS_KEY_ID'],
        aws_secret_access_key=os.environ['SECRET_ACCESS_KEY'] 
    )
    
    # Upload the file
    #for gl in files_gl:  
    fname =  file.replace('.tmp_gl/', '')
    fname = fname.replace(' ', '_').replace('%', '_').replace("'", '_').replace("+", '_').replace('"', '_')
    #s3.upload_file(gl, BUCKET_NAME, fname)
    
    #Read the PDF file in binary mode
    with open(gl, 'rb') as pdf_file:
        s3.put_object(
            Bucket=BUCKET_NAME,
            Key=fname,  # destination in S3
            Body=pdf_file,
            ContentType='application/pdf',
            ACL="public-read"
        )

    return fname
    

def create_excel(data):
    """Create Excel from list of dicts"""
    df = pd.DataFrame(data)
    output = BytesIO()
    with pd.ExcelWriter(output, engine="openpyxl") as writer:
        df.to_excel(writer, index=False, sheet_name="Links")
    output.seek(0)
    return output

# Streamlit UI
st.set_page_config(page_title="File to Link", page_icon="☁️")
st.title("☁️ Upload Multiple Files to Get Excel Links")

uploaded_files = st.file_uploader("Choose files", type=None, accept_multiple_files=True)

if uploaded_files:
    if st.button("Upload All to S3"):
        results = []
        progress_bar = st.progress(0)
        status_text = st.empty()
        total_files = len(uploaded_files)

        for i, file in enumerate(uploaded_files, start=1):
            status_text.text(f"Uploading {file.name} ({i}/{total_files})...")
            url = upload_to_s3(file)
            # if object_name:
            #     url = generate_presigned_url(AWS_BUCKET_NAME, object_name, expiration=3600)
            url = 'https://static-data.gooddoctor.co.id/' + url
            results.append({"Filename": file.name, "Link": url})
            progress_bar.progress(int(i / total_files * 100))
            os.remove(file.name)

        status_text.text("✅ Upload completed!")

        if results:
            st.success(f"{len(results)} files uploaded successfully!")

            # Show results table
            df = pd.DataFrame(results)
            st.dataframe(df)

            # Create Excel for download
            excel_file = create_excel(results)
            st.download_button(
                label="📥 Download Excel with Links",
                data=excel_file,
                file_name="s3_links.xlsx",
                mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
            )
